<a href="https://colab.research.google.com/github/makropolecenia/Book/blob/master/PSKR_LABORAT_HERBERT2_transfer_learning_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#herbert2 09.12.2021 

In [ ]:
#podpiete pliki TSV z PKD train=dev=calosc i mapowanie bez zmian

In [ ]:
### pobiera tokenizer z transformersów z hugging face tylko bez transformersa

In [ ]:
#https://huggingface.co/docs/transformers/master/en/model_doc/herbert#transformers.HerbertTokenizer

# Transfer learning w NLP - dopasuj model do własnego problemu


## AI&NLP Day 2021

### Patryk Pilarski
```
1patryk.pilarski@gmail.com
p.pilarski@sages.com.pl
```

In [ ]:
https://www.nlpday.pl/

In [ ]:
https://stackoverflow.com/questions/57934427/open-ipynb-file-from-the-colab-disk

## Dane
### Allegro Reviews

**Pobranie zbioru**

In [ ]:
# wgranie danych plików do colaba 

!wget https://klejbenchmark.com/static/data/klej_ar.zip
!unzip klej_ar.zip -d klej_ar

--2021-12-09 11:41:46--  https://klejbenchmark.com/static/data/klej_ar.zip
Resolving klejbenchmark.com (klejbenchmark.com)... 35.234.99.58
Connecting to klejbenchmark.com (klejbenchmark.com)|35.234.99.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2314847 (2.2M) [application/zip]
Saving to: ‘klej_ar.zip’

klej_ar.zip         100%[===================>]   2.21M  1.23MB/s    in 1.8s    

2021-12-09 11:41:49 (1.23 MB/s) - ‘klej_ar.zip’ saved [2314847/2314847]

Archive:  klej_ar.zip
replace klej_ar/dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: klej_ar/dev.tsv         
replace klej_ar/test_features.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: klej_ar/test_features.tsv  
replace klej_ar/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: klej_ar/train.tsv       


In [ ]:
# dopisac wczytanie podmienionych danych 

**Wczytanie danych**

In [ ]:
# pliki podmienione muszą być w katalogu klej_ar
# nowy folder w colabie  można utworzyć recznie prawy klawisz myszki utwórz folder 
# wgrać do folderu można recznie i przeciagnac do utworzonego folderu 

In [ ]:
import os
os.listdir("klej_ar")

['train.tsv', 'dev.tsv', '.ipynb_checkpoints']

In [ ]:
with open("klej_ar/train.tsv", "r") as f:
    raw_train = f.readlines()

In [ ]:
with open("klej_ar/dev.tsv", "r") as f:
    raw_dev = f.readlines()

In [ ]:
len(raw_train), len(raw_dev)

(1852, 1852)

**Zbadanie danych**

In [ ]:
raw_train[0]

'text\trating\n'

In [ ]:
raw_train[1]

'Zajmuje się wykonywaniem robót drogowych, w tym prac związanych z budową dróg, placów, chodników oraz parkingów\t1.0\n'

In [ ]:
raw_train[-1]

'Modernizacja tuneli\t15.0'

In [ ]:
labels = []
problematic = []

for doc in raw_train[1:]:
    record = doc.strip().split("\t")
    if len(record) != 2:
        problematic.append(doc)
        continue
    label = record[1]
    labels.append(label)   

In [ ]:
from collections import Counter
Counter(labels)

Counter({'1.0': 100,
         '10.0': 129,
         '11.0': 100,
         '12.0': 138,
         '13.0': 101,
         '14.0': 142,
         '15.0': 94,
         '2.0': 103,
         '3.0': 74,
         '4.0': 100,
         '5.0': 100,
         '6.0': 229,
         '7.0': 138,
         '8.0': 125,
         '9.0': 154})

In [ ]:
len(problematic)

24

In [ ]:
problematic[:5]

['\t8.0\n', '\t8.0\n', '\t8.0\n', '\t8.0\n', '\t8.0\n']

**Przygotowanie danych**

In [ ]:
def prepare_data(raw_data):
    corpus = []
    labels = []
    mapping = {1: 0, 2: 0, 5: 1}
    for doc in raw_data:
        record = doc.strip().split("\t")
        if len(record) != 2:
            continue
        text, target = record
        label = int(float(target))
        if label in mapping: # uproszczenie problemu do klasyfikacji binarnej
            corpus.append(text)
            labels.append(mapping[label])
    return corpus, labels

In [ ]:
train_corpus, train_labels = prepare_data(raw_train[1:])

In [ ]:
test_corpus, test_labels = prepare_data(raw_dev[1:])

In [ ]:
train_data = list(zip(train_corpus, train_labels))
test_data = list(zip(test_corpus, test_labels))

## herBERT

**KLEJ: Comprehensive Benchmark for Polish Language Understanding**   
Piotr Rybak, Robert Mroczkowski, Janusz Tracz, Ireneusz Gawlik

https://www.aclweb.org/anthology/2020.acl-main.111.pdf

https://huggingface.co/transformers/master/model_doc/herbert.html

https://huggingface.co/transformers/master/model_doc/roberta.html?highlight=robertamodel#transformers.RobertaModel

- RobertaModel
- RobertaForCausalLM
- RobertaForMaskedLM
- RobertaForSequenceClassification
- RobertaForMultipleChoice
- RobertaForTokenClassification
- RobertaForQuestionAnswering

In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 29.8 MB/s 
     |████████████████████████████████| 61 kB 486 kB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 


In [ ]:
import random
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import RobertaForSequenceClassification, HerbertTokenizer

torch.manual_seed(42)
random.seed(42)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Tokenizator**

In [ ]:
###### pobiera tokenizer z transformersów z hugging face tylko bez transformersa

In [ ]:
tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMTokenizer'. 
The class this function is called from is 'HerbertTokenizer'.


In [ ]:
train_corpus[0]

'Zajmuje się wykonywaniem robót drogowych, w tym prac związanych z budową dróg, placów, chodników oraz parkingów'

In [ ]:
tokens = tokenizer.tokenize(train_corpus[0])

In [ ]:
tokens

['Zajmuje</w>',
 'się</w>',
 'wykonywaniem</w>',
 'robót</w>',
 'drogowych</w>',
 ',</w>',
 'w</w>',
 'tym</w>',
 'prac</w>',
 'związanych</w>',
 'z</w>',
 'budową</w>',
 'dróg</w>',
 ',</w>',
 'placów</w>',
 ',</w>',
 'chodników</w>',
 'oraz</w>',
 'parking',
 'ów</w>']

In [ ]:
len(tokens)

20

In [ ]:
tokenizer(train_corpus[0])

{'input_ids': [0, 24920, 20, 19054, 4725, 7130, 14, 16, 37, 475, 707, 19, 6019, 1366, 14, 41652, 14, 39204, 35, 18508, 215, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(train_corpus[0], return_tensors="pt")

{'input_ids': tensor([[    0, 24920,    20, 19054,  4725,  7130,    14,    16,    37,   475,
           707,    19,  6019,  1366,    14, 41652,    14, 39204,    35, 18508,
           215,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

**Model**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", 
                                                         num_labels=2, hidden_dropout_prob=0.5, 
                                                         attention_probs_dropout_prob=0.5)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50560, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.5, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
train_corpus[0]

'Zajmuje się wykonywaniem robót drogowych, w tym prac związanych z budową dróg, placów, chodników oraz parkingów'

[Parametry](https://huggingface.co/transformers/master/model_doc/roberta.html?highlight=robertamodel#transformers.RobertaModel.forward)

In [ ]:
outputs = model(**tokenizer(train_corpus[0], return_tensors="pt"))

In [ ]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[-0.1133, -0.0053]], grad_fn=<AddmmBackward0>))])

**Pomocnicze funkcje**

In [ ]:
PAD_TOKEN_ID = tokenizer.pad_token_id

def documents_to_batch(docs, max_len):
    tokenized = tokenizer(docs)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    for i, (inp, att) in enumerate(zip(input_ids, attention_mask)):
        inp_len = len(inp)
        inp = inp[:max_len] + [PAD_TOKEN_ID] * (max_len - inp_len)
        att = att[:max_len] + [0] * (max_len - inp_len)
        input_ids[i], attention_mask[i] = inp, att
    X = torch.LongTensor(input_ids).to(DEVICE)
    ATT = torch.FloatTensor(attention_mask).to(DEVICE)
    return X, ATT

In [ ]:
def train_on_batch(model, optimizer, X, ATT, Y):
    model.train()
    optimizer.zero_grad()
    output = model(input_ids=X, attention_mask=ATT, labels=Y)
    loss = output["loss"]
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
    return loss.item()

In [ ]:
def predict_on_batch(model, X, ATT, Y):
    model.eval()
    output = model(input_ids=X, attention_mask=ATT, labels=Y)
    decision = output["logits"].topk(1).indices.squeeze()
    loss = output["loss"].item()
    equal = decision == Y
    correct = sum(equal).item()
    return correct, decision, loss

**Hiperparametry**

In [ ]:
model = model.to(DEVICE)
learning_rate = 0.000005
epochs = 2
batch_size = 10
max_len = 120
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_train_batches = len(train_data) // batch_size + int(bool(len(train_data) % batch_size))
num_test_batches = len(test_data) // batch_size + int(bool(len(test_data) % batch_size))

best_acc = 0

**Trenowanie modelu**

In [ ]:
for epoch in range(epochs):
    random.shuffle(train_data)
    total_loss = 0
    for n in tqdm(range(num_train_batches)):
        datapoints = train_data[n * batch_size:(n + 1) * batch_size]
        documents, labels = list(zip(*datapoints))
        Y = torch.LongTensor(labels).to(DEVICE)
        X, ATT = documents_to_batch(documents, max_len)
        loss = train_on_batch(model, optimizer, X, ATT, Y)
        total_loss += loss
    print(total_loss)
    with torch.no_grad():
        total = 0
        correct = 0
        dev_loss = 0
        for n in tqdm(range(num_test_batches)):
            datapoints = test_data[n * batch_size:(n + 1) * batch_size]
            documents, labels = list(zip(*datapoints))
            Y = torch.LongTensor(labels).to(DEVICE)
            X, ATT = documents_to_batch(documents, max_len)
            result, _, loss = predict_on_batch(model, X, ATT, Y)
            dev_loss += loss
            total += batch_size
            correct += result
        acc = correct/total * 100
        print(f"acc: {acc}")
        print(f"loss: {dev_loss}")
        if acc > best_acc:
            best_acc = acc
            torch.save(model, "herbert_ar.model")

100%|██████████| 31/31 [00:14<00:00,  2.15it/s]


21.672336220741272


100%|██████████| 31/31 [00:04<00:00,  6.70it/s]


acc: 73.87096774193549
loss: 20.0230171084404


100%|██████████| 31/31 [00:14<00:00,  2.20it/s]


20.673353999853134


100%|██████████| 31/31 [00:04<00:00,  6.63it/s]


acc: 74.51612903225806
loss: 18.6020348072052


In [ ]:
model = torch.load("herbert_ar.model", map_location=DEVICE)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50560, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.5, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
preds = []
for n in tqdm(range(num_test_batches)):
    datapoints = test_data[n * batch_size:(n + 1) * batch_size]
    documents, labels = list(zip(*datapoints))
    Y = torch.LongTensor(labels).to(DEVICE)
    X, ATT = documents_to_batch(documents, max_len)
    _, pred, _ = predict_on_batch(model, X, ATT, Y)
    preds.append(pred)

100%|██████████| 31/31 [00:04<00:00,  6.30it/s]


In [ ]:
preds[:2]

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')]

In [ ]:
preds = [p for t in preds for p in t.tolist()]
documents, labels = list(zip(*test_data))

In [ ]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.74      0.99      0.85       203
           1       0.94      0.30      0.45       100

    accuracy                           0.76       303
   macro avg       0.84      0.65      0.65       303
weighted avg       0.81      0.76      0.72       303



-------

## Zadanie

Przygotuj dane i wytrenuj model na danych **PolEmo2.0-IN**

In [ ]:
!wget https://klejbenchmark.com/static/data/klej_polemo2.0-in.zip
!unzip klej_polemo2.0-in.zip -d klej_polemo2.0in